In [4]:
import cx_Oracle
import pandas as pd
source_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="edp4-scan.cswg.com:1521/EBSF1_USR"
)

In [2]:
query = """select distinct ship_to_site_use_id from APPS.RA_CUSTOMER_TRX_ALL where ship_to_site_use_id is not null
"""

In [6]:
%%time
df= pd.read_sql_query(query, source_conn)
df

Wall time: 1min 24s


,SHIP_TO_SITE_USE_ID
0,118414
1,17766
2,18273
3,18391
4,18437
...,...
24441,120839
24442,122104
24443,122396
24444,48689


In [10]:
%%time
# Initialize a variable to store the total count of updated rows
total_updated_rows = 0
# Iterate through unique ship_to_site_id values in the DataFrame
for ship_to_site_id in df['SHIP_TO_SITE_USE_ID'].unique():
    # Construct and execute the SQL update statement for each ship_to_site_id
    update_query = f"""
        UPDATE xxfin.xxar_customer_master a
        SET a.last_bill_date = (SELECT MAX(TRX_DATE) FROM APPS.RA_CUSTOMER_TRX_ALL b WHERE b.SHIP_TO_SITE_USE_ID = {ship_to_site_id})
        WHERE a.SHIP_TO_SITE_USE_ID = {ship_to_site_id}
    """

    
    # Create a cursor and execute the update statement
    with source_conn.cursor() as cursor:
        cursor.execute(update_query)
    
    # Commit the changes to the database
#     connection.commit()
        updated_rows = cursor.rowcount
        total_updated_rows += updated_rows

        # Print the count for the current iteration
#         print(f"Updated {updated_rows} rows for ship_to_site_id: {ship_to_site_id}")

        # Print the total count of updated rows
print(f"Total updated rows: {total_updated_rows}")
# Close the database connection
connection.close()

DatabaseError: DPI-1010: not connected